In [2]:
from astropy.table import Table
from operator import itemgetter
from skynet.api.v20 import client
from skynet.api.v20.methods import download, groups, obs, scopes


In [3]:
group_id = groups.get("Majewski Research Group")["id"]
scope_id = scopes.query(include="*", **{"name": "RRRT"})[0]["id"]
group_id, scope_id

(816, 27)

In [8]:
obs.query(include="*", **{"id": 3702107})


[{'triggerRepointEnabled': False,
  'decDegs': 2.61905638888889,
  'timeIn': '2019-03-08T19:36:11',
  'user': {'username': 'meb4bc', 'id': 25103},
  'objectType': 'sidereal',
  'group': {'name': 'Majewski Research Group', 'id': 816},
  'priority': 1,
  'constantDecOffsetArcmins': 0.0,
  'trigger': None,
  'orbitalElementsId': None,
  'teleOwnerId': 17,
  'telescopes': [{'name': 'RRRT',
    'siteName': 'FMO',
    'site': {'lngDegs': -78.6941167,
     'name': 'FMO',
     'latDegs': 37.879083,
     'id': 17},
    'id': 27}],
  'minEl': 30.0,
  'ditherEnabled': False,
  'obsNotes': [],
  'name': 'K2-22b',
  'parentCollabTimeAccountId': None,
  'parentGroupTimeAccountId': 43881,
  'targetTracking': 'track_target',
  'fieldLockUtc': None,
  'groupId': 816,
  'timeSubmittedUtc': '2019-03-08T19:36:11',
  'exps': [{'linkedExps': [],
    'commandedDec': 2.61905638888889,
    'timeIn': '2019-03-08T19:36:11',
    'filterName': 'R',
    'filterIdUsed': 9,
    'filterRequested': {'name': 'R', 'id': 

In [16]:
# fields = "*"
observation_fields = {
    "id": "Observation ID",
    "state": "Observation state",
    "priority": "Priority",
    "name": "Target Name",
    "user.username": "Username",
    "group.name": "Group",
    "raHours": "RA",
    "decDegs": "Dec",
    "currentTelescope.name": "Telescope",
    "teleOwner.name": "Scope Owner",
    "timeIn": "Submitted",
}

observation_criteria = {"groupId": 816}
# criteria = {"id": 3702107}
observation_results = obs.query(include=",".join(observation_fields.keys()), **observation_criteria)

observations = Table()
for field, col_name in observation_fields.items():
    field_parts = field.split(".")
    match len(field_parts):
        case 1:
            observations[col_name] = list(map(itemgetter(field), observation_results))
        case 2:
            field_dicts = list(map(itemgetter(field_parts[0]), observation_results))
            observations[col_name] = [field_dict[field_parts[1]] if field_dict is not None else "" for field_dict in field_dicts]
observations.write("observations.csv", overwrite=True)

observations

Observation ID,Observation state,Priority,Target Name,Username,Group,RA,Dec,Telescope,Scope Owner,Submitted
int32,str9,int32,str29,str9,str23,float64,float64,str4,str3,str19
3702107,completed,1,K2-22b,meb4bc,Majewski Research Group,11.2988547222222,2.61905638888889,RRRT,UVA,2019-03-08T19:36:11
7474102,canceled,7,1194_01,mharris,Majewski Research Group,11.188,69.96444444444445,,UVA,2022-03-11T21:48:04
7474104,canceled,7,1251_01,mharris,Majewski Research Group,18.23525,62.85805555555556,,UVA,2022-03-11T22:43:08
7474105,canceled,7,1775_01,mharris,Majewski Research Group,10.007666666666667,39.45777777777778,,UVA,2022-03-11T22:52:48
7484500,completed,7,1775_01,mharris,Majewski Research Group,10.007666666666667,39.45777777777778,RRRT,UVA,2022-03-14T15:18:47
7505658,canceled,7,1251_01,mharris,Majewski Research Group,18.23525,62.85805555555556,RRRT,UVA,2022-03-19T01:24:24
7514494,canceled,7,2017_01,mharris,Majewski Research Group,15.263333333333334,27.361666666666668,RRRT,UVA,2022-03-21T02:09:05
7514505,canceled,7,1295_01,mharris,Majewski Research Group,18.24666666666667,67.87138888888889,RRRT,UVA,2022-03-21T03:22:16
7514506,canceled,7,1273_01,mharris,Majewski Research Group,14.274694444444446,58.390277777777776,,UVA,2022-03-21T03:22:16


In [17]:
exposure_fields = {
    "exps.obsId": "Observation ID",
    "exps.expNum": "Exposure Num",
    "exps.imageId": "Image ID",
    "exps.obs.name": "Name",
    "exps.type": "Type",
    "exps.filterName": "Filter Name",
    "exps.timeTaken": "Time Taken",
    "exps.state": "Image State",
    "exps.expLengthUsed": "Exposure",
    "exps.binningUsed": "Binning",
    "exps.gain": "Gain",
    "exps.camera.name": "Camera",
}

# criteria = {"groupId": 816}
# exposure_criteria = {"id": 3702107}
exposure_criteria = {"id": 10408766}

exposure_results = obs.query(include=",".join(exposure_fields.keys()), **exposure_criteria)

exp_struct = [result["exps"] for result in exposure_results][0]
images = Table()
for field, col_name in exposure_fields.items():
    field_parts = field.split(".")
    match len(field_parts):
        case 2:
            images[col_name] = list(map(itemgetter(field_parts[1]), exp_struct))
        case 3:
            field_dicts = list(map(itemgetter(field_parts[1]), exp_struct))
            images[col_name] = [field_dict[field_parts[2]] if field_dict is not None else "" for field_dict in field_dicts]
images.write("images.csv", overwrite=True)

images

Observation ID,Exposure Num,Image ID,Name,Type,Filter Name,Time Taken,Image State,Exposure,Binning,Gain,Camera
int32,int32,str9,str17,str5,str1,object,str8,object,int32,object,str40
10408766,0,r39590217,TIC 367448265 Bb,light,V,2023-12-09T05:01:08,archived,1.0,2,1.32536,UVA SBIG STX-16803 with FLI Filter Wheel
10408766,1,r39590218,TIC 367448265 Bb,light,V,2023-12-09T05:01:17,archived,1.0,2,1.32536,UVA SBIG STX-16803 with FLI Filter Wheel
10408766,2,r39590219,TIC 367448265 Bb,light,V,2023-12-09T05:01:26,archived,1.0,2,1.32536,UVA SBIG STX-16803 with FLI Filter Wheel
10408766,3,r39590220,TIC 367448265 Bb,light,V,2023-12-09T05:01:34,archived,1.0,2,1.32536,UVA SBIG STX-16803 with FLI Filter Wheel
10408766,4,r39590221,TIC 367448265 Bb,light,V,2023-12-09T05:01:43,archived,1.0,2,1.32536,UVA SBIG STX-16803 with FLI Filter Wheel
10408766,5,r39590222,TIC 367448265 Bb,light,V,2023-12-09T05:01:51,archived,1.0,2,1.32536,UVA SBIG STX-16803 with FLI Filter Wheel
10408766,6,r39590223,TIC 367448265 Bb,light,V,2023-12-09T05:02:00,archived,1.0,2,1.32536,UVA SBIG STX-16803 with FLI Filter Wheel
10408766,7,r39590224,TIC 367448265 Bb,light,V,2023-12-09T05:02:08,archived,1.0,2,1.32536,UVA SBIG STX-16803 with FLI Filter Wheel
10408766,8,r39590225,TIC 367448265 Bb,light,V,2023-12-09T05:02:17,archived,1.0,2,1.32536,UVA SBIG STX-16803 with FLI Filter Wheel


In [36]:
asd, qwe = download.fits(**{"obs": "10408766"})
# asd, qwe = download.fits(**{"image": "r34306388"})

BadRequestError: No data to return

In [16]:
asd = obs.query(include="exps", **{"id": 3702107})[0]["exps"]
# asd = obs.query(include="exps", **{"id": 7474102})
list(map(lambda x: x["imageId"], filter(lambda x: x["state"] in ["ready", "completed", "archived"], asd)))
# asd


['r24092276',
 'r24092277',
 'r24092278',
 'r24092279',
 'r24092280',
 'r24092281',
 'r24092282',
 'r24092283',
 'r24092284',
 'r24092285',
 'r24092286',
 'r24092287']